In [353]:
from collections import namedtuple
import time
import sys
import numpy as np
from heapq import heappush

class Edge:
    def __init__(self, origin=None, destination=None):
        self.origin = origin
        self.weight = 1
        self.destination = destination

    def __repr__(self):
        return f"edge: {self.origin} {self.weight} {self.destination}"


class Airport:
    def __init__(self, iden=None, name=None):
        self.code = iden
        self.name = name
        self.routes = []
        self.routeHash = dict()
        self.outweight = 0
        self.pageIndex = 0

    def __repr__(self):
        return f"{self.code}\t{self.outweight}\t{self.name}"

    def add_destination(self, destination_airport, weight=1):
        if destination_airport.code in self.routeHash:
            self.routeHash[destination_airport.code] += weight
        else:
            self.routes.append(destination_airport)
            self.routeHash[destination_airport.code] = weight

        self.outweight += weight


edgesHash = dict()
airportList = []
airportHash = dict()

def readAirports(fd):
    print("Reading Airport file from {0}".format(fd))
    airportsTxt = open(fd, "r", encoding="utf-8")
    cont = 0
    for line in airportsTxt.readlines():
        a = Airport()
        try:
            temp = line.split(',')
            if len(temp[4]) != 5:
                raise Exception('not an IATA code')
            
            code = temp[4][1:-1]
            # check for valid codes
            if len(code) == 3 and code.isalpha():
                a.code = code
                a.name = temp[1][1:-1] + ", " + temp[3][1:-1]
                cont += 1
                airportList.append(a)
                airportHash[a.code] = a
            else:
                pass

        except Exception:
            pass

    airportsTxt.close()
    print(f"There were {cont} Airports with IATA code")

def readRoutes(fd):
    print(f"Reading Routes file from {fd}")
    routesTxt = open(fd, "r", encoding="utf-8")
    missing_IATA_code = {"origin": [], "destination": []}
    for line in routesTxt.readlines():
        try:
            temp = line.split(',')
            origin = temp[2]
            destination = temp[4]

            if len(origin) == 3 and origin.isalpha() and len(destination) == 3 and destination.isalpha():
                if origin not in airportHash:
                    missing_IATA_code["origin"].append(destination)
                    new_airport = Airport(origin, "NaN")
                    airportHash[origin] = new_airport
                    airportList.append(new_airport)
                elif destination not in airportHash:
                    missing_IATA_code["destination"].append(origin)
                    new_airport = Airport(destination, "NaN")
                    airportHash[destination] = new_airport
                    airportList.append(new_airport)

                airportHash[origin].add_destination(airportHash[destination])
        except Exception:
            pass

    print(f"Sink nodes: {len(missing_IATA_code['origin'])} for origin and {len(missing_IATA_code['destination'])} destination")


def computePageRanks(airports_list, airport_hash, epsilon=1e-10, L=0.8, max_iter=5000):
    time1 = time.time()
    n = len(airports_list)
    
    # Initialize PageRank values (uniform distribution initially)
    P = np.ones(n) / n
    teleport = (1 - L) / n

    # Create an empty heap to store (page rank, airport code) pairs
    heap = []

    # Iterate for a maximum of `max_iter` iterations or until convergence
    for iter in range(max_iter):
        Q = np.zeros(n)  # Create a new array to store PageRank values for this iteration
        sink_weight = 0

        # Analyze number of sink nodes and calculate their weights
        for j in range(n):
            airport = airports_list[j]
            
            if airport.outweight == 0:
                # Sink node: no outgoing flights, accumulate its PageRank
                sink_weight += airport.pageIndex

        # Calculate new PageRank for each airport
        for j in range(n):
            airport = airports_list[j]
            link_sum = 0

            if airport.outweight != 0:
                # For airports with outgoing flights, accumulate weighted contributions from all outbound airports
                for code, weight in airport.routeHash.items():
                    linked_airport = airport_hash[code]
                    link_sum += ((linked_airport.pageIndex * weight) / airport.outweight) + (sink_weight / n)
            else:
                # For sink nodes, distribute its PageRank equally among all airports
                link_sum += sink_weight / n

            # Calculate the new PageRank for airport `j`
            Q[j] = L * link_sum + teleport

            # Push (page rank, airport code) to the heap with negative rank for max-heap behavior
            heappush(heap, (-Q[j], airport.code))

        # Check for convergence based on L1 norm (difference between new and old PageRank)
        if np.linalg.norm(Q - P, 1) < epsilon:
            time2 = time.time()
            print(f"Converged after {iter + 1} iterations and {round(time2 - time1, 3)} s")
            # After convergence, return the sorted heap
            return heap

        # Update PageRank vector for the next iteration
        P = Q
    
    time2 = time.time()
    print(f"Reached max iterations: {max_iter}. Time {round(time2 - time1, 3)} s")
    return heap

def outputPageRanks(sorted_list, output_filename):
    with open(output_filename, 'w', encoding='utf-8') as file:
        for rank, airport in sorted_list:
            file.write(f"{-rank:.10f}\t{airport}\n")

#### Execute in main ####
readAirports("airports_test.txt")
readRoutes("routes_test.txt")


# Update weights for each airport so that they are constant
time1 = time.time()
for airport in airportList:
    airport.pageIndex = 1/len(airportList)

sorted_weights = computePageRanks(airportList, airportHash)

outputPageRanks(sorted_weights, output_filename = "sorted_weights.txt")

Reading Airport file from airports_test.txt
There were 2 Airports with IATA code
Reading Routes file from routes_test.txt
Sink nodes: 0 for origin and 0 destination
Converged after 1 iterations and 0.0 s


In [350]:
# check weights sum 1
weights = [w[0] for w in sorted_weights]
sum(weights)

np.float64(1.0)

In [352]:
weights

[np.float64(0.5), np.float64(0.5)]